In [2]:
import pandas as pd
import json
import numpy as np
import datetime


In [3]:
def handle_price(price):
    try:
        return float(price)
    except ValueError:
        return np.nan

In [4]:
# Pandas settings
# representing large numbers properly
pd.options.display.float_format = '{:.2f}'.format


In [5]:
FILE_PATH='/home/rizki/fun/car-analysis/datasets/output-grande.json'
df = pd.read_json(FILE_PATH)

In [6]:
# Handling price data
df['Price'] = df['Price'].apply(handle_price)
df['priceNotes'] = df['Price'].apply(lambda x: 'Call for price' if pd.isnull(x) else '')
cols = df.columns.tolist()
cols.insert(cols.index('Price') + 1, cols.pop(cols.index('priceNotes')))
df = df[cols]

In [7]:
# Converting data types
df['Ad No'] = df['Ad No'].astype(str)
df['Name'] = df['Name'].astype(str)
df['Price'] = df['Price'].astype(float)
df['Model Year'] = pd.to_datetime(df['Model Year'], format='%Y').dt.year
df['Mileage'] = df['Mileage'].astype(int)
df['Engine Type'] = df['Engine Type'].astype(str)
df['Engine Capacity'] = df['Engine Capacity'].astype(str)
df['Transmission'] = df['Transmission'].astype(str)
df['Color'] = df['Color'].astype(str)
df['Assembly'] = df['Assembly'].astype(str)
df['Body Type'] = df['Body Type'].astype(str)
df['Features'] = df['Features'].astype(str)
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df['URL'] = df['URL'].astype(str)

In [8]:
# Process features -- add each item as column
# Process the "Features" column
features_df = df['Features'].str.get_dummies(sep=', ')
df = pd.concat([df, features_df], axis=1)

# Drop the original 'Features' column
df = df.drop(columns=['Features'])

## Explorations

In [9]:
df.head(3)

,Ad No,Name,Price,priceNotes,Model Year,Location,Mileage,Registered City,Engine Type,Engine Capacity,...,Power Locks,Power Mirrors,Power Steering,Power Windows,Rear AC Vents,Rear Camera,Rear Speakers,Steering Switches,Sun Roof,USB and Auxillary Cable
0,7548009,Toyota Corolla Altis Grande CVT-i 1.8 2020,5750000.00,,2020,\n,48300,NaN,Petrol,1798 cc,...,1,1,1,1,0,0,0,0,1,0
1,7547540,Toyota Corolla Altis Grande X CVT-i 1.8 Beige ...,7500000.00,,2022,\n,600,NaN,Petrol,1799 cc,...,1,1,1,1,0,0,0,0,1,0
2,7499742,Toyota Corolla Altis Grande X CVT-i 1.8 Black ...,5995000.00,,2021,\n,22000,NaN,Petrol,1798 cc,...,1,1,1,1,0,0,0,0,1,0


In [10]:
# Summary Statistics
df.describe()

,Price,Model Year,Mileage,Registered City,Last Updated,ABS,AM/FM Radio,Air Bags,Air Conditioning,AM/FM Radio,...,Power Locks,Power Mirrors,Power Steering,Power Windows,Rear AC Vents,Rear Camera,Rear Speakers,Steering Switches,Sun Roof,USB and Auxillary Cable
count,411.00,426.00,426.00,0.00,426,426.00,426.00,426.00,426.00,426.00,...,426.00,426.00,426.00,426.00,426.00,426.00,426.00,426.00,426.00,426.00
mean,5608442.82,2018.41,47691.58,NaN,2023-05-26 04:30:25.352112640,0.97,0.02,0.00,0.00,0.97,...,0.98,0.99,0.99,0.99,0.01,0.10,0.11,0.10,0.92,0.11
min,1365000.00,1993.00,38.00,NaN,2023-04-28 00:00:00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,4350000.00,2017.00,16000.00,NaN,2023-05-18 00:00:00,1.00,0.00,0.00,0.00,1.00,...,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
50%,5800000.00,2019.50,40000.00,NaN,2023-05-30 00:00:00,1.00,0.00,0.00,0.00,1.00,...,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
75%,6900000.00,2021.00,71000.00,NaN,2023-06-06 00:00:00,1.00,0.00,0.00,0.00,1.00,...,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00
max,8000000.00,2022.00,200000.00,NaN,2023-06-11 00:00:00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
std,1471532.22,4.29,38048.38,NaN,NaN,0.17,0.13,0.05,0.05,0.18,...,0.13,0.11,0.10,0.12,0.10,0.30,0.31,0.30,0.27,0.31


In [11]:
# Metric definitions
"""
    AgeScore:
    
    This score represents the relative 'newness' 
    of the car. A higher value means the car is 
    newer relative to the maximum car age considered.

    MileageScore:
    
    This score represents the relative amount of 
    use the car has had. A higher score means the car 
    has lower mileage relative to the maximum mileage considered.

    PriceScore: 
    
    This score represents the relative affordability of the car. 
    A higher score means the car is cheaper relative to the maximum 
    price considered.

    CarValueScore: 
    
    This score is an average of the Age, Mileage, and Price scores. 
    It provides an overall value score for the car considering these 
    three factors, with a higher score indicating better value.
    
"""
# Add columns with metrics
current_year = datetime.datetime.now().year
MaxCarAge = 20  # adjust as per your definition of an 'old' car
MaxMileage = 200000  # adjust as per your definition of 'high-mileage'
MaxPrice = 30000  # adjust as per the maximum price you are willing to pay

df['AgeScore'] = 1 - ((current_year - df['Model Year']) / MaxCarAge)
df['MileageScore'] = 1 - (df['Mileage'] / MaxMileage)
df['PriceScore'] = 1 - (df['Price'] / MaxPrice)

df['CarValueScore'] = (df['AgeScore'] + df['MileageScore'] + df['PriceScore']) / 3

In [12]:
df.describe()

,Price,Model Year,Mileage,Registered City,Last Updated,ABS,AM/FM Radio,Air Bags,Air Conditioning,AM/FM Radio,...,Rear AC Vents,Rear Camera,Rear Speakers,Steering Switches,Sun Roof,USB and Auxillary Cable,AgeScore,MileageScore,PriceScore,CarValueScore
count,411.00,426.00,426.00,0.00,426,426.00,426.00,426.00,426.00,426.00,...,426.00,426.00,426.00,426.00,426.00,426.00,426.00,426.00,411.00,411.00
mean,5608442.82,2018.41,47691.58,NaN,2023-05-26 04:30:25.352112640,0.97,0.02,0.00,0.00,0.97,...,0.01,0.10,0.11,0.10,0.92,0.11,0.77,0.76,-185.95,-61.47
min,1365000.00,1993.00,38.00,NaN,2023-04-28 00:00:00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,-0.50,0.00,-265.67,-87.91
25%,4350000.00,2017.00,16000.00,NaN,2023-05-18 00:00:00,1.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.70,0.65,-229.00,-75.70
50%,5800000.00,2019.50,40000.00,NaN,2023-05-30 00:00:00,1.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.82,0.80,-192.33,-63.56
75%,6900000.00,2021.00,71000.00,NaN,2023-06-06 00:00:00,1.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.90,0.92,-144.00,-47.56
max,8000000.00,2022.00,200000.00,NaN,2023-06-11 00:00:00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,0.95,1.00,-44.50,-14.78
std,1471532.22,4.29,38048.38,NaN,NaN,0.17,0.13,0.05,0.05,0.18,...,0.10,0.30,0.31,0.30,0.27,0.31,0.21,0.19,49.05,16.24


In [13]:
df.to_csv('/home/rizki/fun/car-analysis/datasets/2023-06-12-cleaned-data.csv')